<a href="https://colab.research.google.com/github/s1scottd/CIFAR-10_Image-Classification-CNN/blob/main/CIFAR_10_Image_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries. 

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from keras.datasets import mnist
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from keras.datasets import cifar10

##Prepare the data

Define a reshape and normalize function

In [ ]:
def reshape_and_normalize(data):
  shape = data.shape
  data = np.reshape(data,(shape[0],32,32,3))
  data = data/255.
  return data

Load, Reshape and Normalize the CIFR-10 Dataset

In [ ]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train[:-10000]
X_val = X_train[-10000:]
y_train = y_train[:-10000]
y_val = y_train[-10000:]

X_train = reshape_and_normalize(X_train)
X_test = reshape_and_normalize(X_test)
X_val = reshape_and_normalize(X_val)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape:  {X_test.shape}")
print(f"X_val shape: {X_val.shape}")

# Convolutional Neural Network without Tuning

Define and compile the CNN model

In [ ]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10)
])

# Compile the model
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

Fit the CNN model

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Evaluate the model

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('Test accuracy:', test_acc)

# Convolutional Neural Network with Tuning

Install and import Tuner

In [ ]:
!pip install keras-tuner --upgrade
import keras_tuner

Create a tunable model

In [ ]:


def build_model(hp):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(
        # tune number of units
        units = hp.Int('units', min_value=32, max_value=512, step=32),
        # tune the activation function to use
        activation=hp.Choice("activation", ["relu", "tanh"])))
  model.add(Dense(10, activation="softmax"))
  
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
  
  return model

Build and Compile the model

In [ ]:
build_model(keras_tuner.HyperParameters())

Create the tuner

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective='accuracy',
    max_trials=10,
    executions_per_trial=5)

Execute a search for the best model

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
best_model = tuner.get_best_models()[0]
print(f"Search Space Summary:\n{tuner.search_space_summary}")